In [ ]:
import sys
sys.path.append('../craw-data')
from kafka import KafkaProducer
import requests
from json import dumps
import time
from kafka import KafkaConsumer

In [ ]:
from get_detaill_item import get_detail_product
from get_item_and_shop_ids import get_item_shop_info

In [ ]:
import pandas as pd
categoryList = pd.read_csv('../craw-data/raw-data/category_list.csv')
categoryList.head()

In [ ]:
def get_data(raw_dict, list_attribute):
    output = {}
    for att in list_attribute:
        output[att] = raw_dict[att]
    return output

In [ ]:
from kafka import KafkaProducer
kafka_producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda x: dumps(x).encode('utf-8'))
# from a - b category
a = 24
b = 27
list_attributes = ["itemid", "shopid", "price_before_discount", "price", "discount", "historical_sold", "sold", "name", "item_status", "item_rating", "liked_count", "cmt_count", "item_type", "shop_location", "global_sold", "shop_vouchers", "categories"]
for item in range(a,b):
    data = []
    topic = categoryList.catid.iloc[item]
    print(f'topic {item} "{categoryList.display_name.iloc[item]}" is runing...')
    for newest in range(0,8001,60): 
        item_list_id = get_item_shop_info(topic,newest)
        data.extend(get_detail_product(item_list_id,topic))
    message_data = list(map(lambda x: get_data(x,list_attributes),data))
    for it in message_data:
        print("publishing to kafka...")
        kafka_producer.send('item-shopee', value=it)